In [ ]:
# Utilizando o -q para apareccer menos informações
#pip install langchain
#pip install langdetect
#pip install tiktoken
#pip install gradio
#pip install pypdf
#pip install openai
#pip install chromadb
#pip install pypdf -q

In [4]:
# from Bibliotecas ou Pacotes . Módulo import Classe 

# Importando as classes que usaremos

import os                                                                 # Biblioteca para manipular o SO
from dotenv import load_dotenv, find_dotenv                               # Carregar arquivo .env
from langchain.document_loaders import PyPDFLoader                        # Carregar o PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter        # Separar as palavras
from langchain.vectorstores import Chroma                                 # Banco de Dados de Vetores
from langchain.embeddings.openai import OpenAIEmbeddings                  # Embedding das palavras
from langchain.chains import RetrievalQA                                  # Perguntas e Respostas
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI                              # Modelo utilizado
import gradio as gr                                                       # Criar o chatbot

In [5]:
load_dotenv(find_dotenv(), override=True)

True

### Backend

In [6]:
class PerguntasRespostas():
    
    #def __init__(self, load=None, chunk_size=1000, chunk_overlap=0, arquivo=None, split=None, vector_store=None):
    def __init__(self, arquivo=None, documento=None, text_splitter=None, all_qa=None):
        self.arquivo = arquivo
        self.documento = documento
        self.text_splitter = text_splitter
        self.chunk_size = 1000
        self.chunk_overlap = 0
        #self.all_split = all_split
        #self.all_embedding = all_embedding
        self.all_qa = all_qa
        
    def chunk(self, documento):
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size, 
                                                            chunk_overlap=self.chunk_overlap)
        split = self.text_splitter.split_documents(documento) 
        return split

    def embeddings(self, split):
        embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')  
        vector_store = Chroma.from_documents(split, embeddings)
        return vector_store
    
    def qa(self, vector_store):
        llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.1)
        retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3}) 
        chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, 
                                            return_source_documents=True) 
        return chain
            
    def carregar(self, arquivo):
        try:
            try:
                loader = PyPDFLoader(arquivo)
                documento = loader.load()
            except Exception as e:
                print('Erro no Carregamento')
                print(e)
            
            #Chunk
            try:
                self.all_split = self.chunk(documento)
            except Exception as e:
                print('Erro no split')
                print(e)
            
            #Embeddings
            try:
                self.all_embedding = self.embeddings(self.all_split)
            except Exception as e:
                print('Erro no embedding')
                print(e)
            
            #QA
            try:
                self.all_qa = self.qa(self.all_embedding)
            except Exception as e:
                print('Erro no qa')
                print(e)
                
        except Exception as e:
            print(e)
            
    def resposta(self, query):

        if self.all_qa is not None:
            result = self.all_qa(query)
            result = result['result']
        else:
            result = 'Digite sua pergunta'
       
        return result

In [7]:
question = PerguntasRespostas()

In [8]:
question.carregar('unicapagina.pdf')

In [10]:
question.resposta('Do que trata esse pdf?')

'Esse PDF é um discurso proferido por Winston Churchill em 4 de junho de 1940, na Câmara dos Comuns. O discurso é intitulado "Lutaremos nas praias" e aborda a situação da Segunda Guerra Mundial, especificamente a retirada das tropas britânicas e francesas de Dunkirk. O discurso fala sobre a estratégia militar, a resistência das tropas britânicas e francesas e a determinação em continuar lutando contra a Alemanha nazista.'

### Frontend